In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
model_name =  "gpt-4o-mini" # gpt-4o-mini or gpt-4o

print(f'Project Connection String: <...{os.environ["PROJECT_CONNECTION_STRING"][-30:]}>')

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


In [2]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai04-rg',
 'project_name': 'mmai-hub04-prj01-fvye'}

In [3]:
from azure.ai.projects.models import CodeInterpreterTool

code_interpreter = CodeInterpreterTool()

print(f"Code interpreter definitions: {code_interpreter.definitions}")
print(f"Code interpreter resources: {code_interpreter.resources}")
print(f"Code interpreter file_ids: {code_interpreter.file_ids}")

Code interpreter definitions: [{'type': 'code_interpreter'}]
Code interpreter resources: {}
Code interpreter file_ids: set()


In [4]:
agent = project_client.agents.create_agent(
    model=model_name,
    name="my-agent",
    instructions="You are a helpful agent",
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources
)

agent.items

<bound method _MyMutableMapping.items of {'id': 'asst_CR3B5kU5skX3Yelakjd4Q9Xu', 'object': 'assistant', 'created_at': 1735635748, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}>

In [5]:
# Create a second agent
agent_02 = project_client.agents.create_agent(
    model=model_name,
    name="my-agent",
    instructions="You are a helpful agent",
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources,
)

agent_02.items

<bound method _MyMutableMapping.items of {'id': 'asst_GfTYb6cmqahxeBItG8XihPh9', 'object': 'assistant', 'created_at': 1735635749, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}>

In [6]:
# List all agents
print(project_client.agents.list_agents()['data'])

[{'id': 'asst_GfTYb6cmqahxeBItG8XihPh9', 'object': 'assistant', 'created_at': 1735635749, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}, {'id': 'asst_CR3B5kU5skX3Yelakjd4Q9Xu', 'object': 'assistant', 'created_at': 1735635748, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}]


In [7]:
project_client.agents.delete_agent(agent_02.id)

{'id': 'asst_GfTYb6cmqahxeBItG8XihPh9', 'object': 'assistant.deleted', 'deleted': True}

In [8]:
# Create a thread
thread = project_client.agents.create_thread()

print(f"Created thread: {thread}")

Created thread: {'id': 'thread_mZQ5xiCwTDp4QEG5Sa0wJUJ3', 'object': 'thread', 'created_at': 1735635750, 'metadata': {}, 'tool_resources': {}}


In [9]:
# Create a message
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="""
    Could you please create a bar chart for the operating profit using the following data and provide the file to me? 
    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, Company D: $1.8 million
    """,
)
print(f"Created message: {message}")

Created message: {'id': 'msg_nljCtluKiV7VKTZQk0kiVRJV', 'object': 'thread.message', 'created_at': 1735635750, 'assistant_id': None, 'thread_id': 'thread_mZQ5xiCwTDp4QEG5Sa0wJUJ3', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': '\n    Could you please create a bar chart for the operating profit using the following data and provide the file to me? \n    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, Company D: $1.8 million\n    ', 'annotations': []}}], 'attachments': [], 'metadata': {}}


In [10]:
%%time

# Run the agent
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)

print(f"Run status: {run.status}.\nRun details: {run}")

Run status: RunStatus.COMPLETED.
Run details: {'id': 'run_XUKVyWG4IyFg24achz80Quk8', 'object': 'thread.run', 'created_at': 1735635751, 'assistant_id': 'asst_CR3B5kU5skX3Yelakjd4Q9Xu', 'thread_id': 'thread_mZQ5xiCwTDp4QEG5Sa0wJUJ3', 'status': 'completed', 'started_at': 1735635751, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1735635759, 'required_action': None, 'last_error': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': 'auto', 'last_messages': None}, 'incomplete_details': None, 'usage': {'prompt_tokens': 407, 'completion_tokens': 218, 'total_tokens': 625}, 'response_format': 'auto', 'tool_choice': 'auto', 'parallel_tool_calls': True}
CPU times: total: 31.2 ms
Wall time: 10.5 s


In [11]:
# Get messages from the thread
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

Messages: {'object': 'list', 'data': [{'id': 'msg_ed5H0KFV5tN5YMLTVdst49Dt', 'object': 'thread.message', 'created_at': 1735635759, 'assistant_id': 'asst_CR3B5kU5skX3Yelakjd4Q9Xu', 'thread_id': 'thread_mZQ5xiCwTDp4QEG5Sa0wJUJ3', 'run_id': 'run_XUKVyWG4IyFg24achz80Quk8', 'role': 'assistant', 'content': [{'type': 'image_file', 'image_file': {'file_id': 'assistant-95T2W29SgBJlPXNSj9ksqKUC'}}, {'type': 'text', 'text': {'value': 'I have created the bar chart for the operating profit. You can download it using the link below:\n\n[Download Operating Profit Chart](sandbox:/mnt/data/operating_profit_chart.png)', 'annotations': [{'type': 'file_path', 'text': 'sandbox:/mnt/data/operating_profit_chart.png', 'start_index': 132, 'end_index': 176, 'file_path': {'file_id': 'assistant-uCz4zerwHXk9xYbaE1jmf3nH'}}]}}], 'attachments': [{'file_id': 'assistant-uCz4zerwHXk9xYbaE1jmf3nH', 'tools': [{'type': 'code_interpreter'}]}], 'metadata': {}}, {'id': 'msg_nljCtluKiV7VKTZQk0kiVRJV', 'object': 'thread.messag

In [12]:
# Get the last message from the sender
last_msg = messages.get_last_text_message_by_sender("assistant")
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

Last Message: I have created the bar chart for the operating profit. You can download it using the link below:

[Download Operating Profit Chart](sandbox:/mnt/data/operating_profit_chart.png)


In [13]:
# messages.file_path_annotations[0].text.split('/')[-1]
messages.file_path_annotations

[{'type': 'file_path', 'text': 'sandbox:/mnt/data/operating_profit_chart.png', 'start_index': 132, 'end_index': 176, 'file_path': {'file_id': 'assistant-uCz4zerwHXk9xYbaE1jmf3nH'}}]

In [14]:
for file_path_annotation in messages.file_path_annotations:
    print(f"File Paths: {file_path_annotation}")
    file_name = file_path_annotation.text.split('/')[-1]
    project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name)
    print(f"Saved text file to: {os.getcwd()}\\{file_name}")

File Paths: {'type': 'file_path', 'text': 'sandbox:/mnt/data/operating_profit_chart.png', 'start_index': 132, 'end_index': 176, 'file_path': {'file_id': 'assistant-uCz4zerwHXk9xYbaE1jmf3nH'}}
Saved text file to: E:\Users\mauromi\source\git_repos\aaas\operating_profit_chart.png


In [15]:
# Generate an image file for the bar chart
for image_content in messages.image_contents:
    print(f"Image content: {image_content}")
    file_name = f"{image_content.image_file.file_id}_image_file.png"
    project_client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
    print(f"Saved image file to: {os.getcwd()}\\{file_name}")

Image content: {'type': 'image_file', 'image_file': {'file_id': 'assistant-95T2W29SgBJlPXNSj9ksqKUC'}}
Saved image file to: E:\Users\mauromi\source\git_repos\aaas\assistant-95T2W29SgBJlPXNSj9ksqKUC_image_file.png


In [16]:
# Delete all agents
for pca in project_client.agents.list_agents()['data']:
    print(f"Deleting agent {pca.name} ({pca.id})...")
    project_client.agents.delete_agent(pca.id)

Deleting agent my-assistant (asst_bUARfrd9ZQ8RKKYWvRYydUUh)...
Deleting agent my-agent (asst_CR3B5kU5skX3Yelakjd4Q9Xu)...


# HIC SUNT LEONES